In [1]:
from time import sleep

import nltk
import requests
import bs4
import re
import numpy as np
import pandas as pd
from nltk import SnowballStemmer


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LinearRegression, SGDClassifier
from sklearn.svm import LinearSVC

In [42]:
links = []
for page in range(1,19):
# for page in range(1,50):
    url = f'https://www.dns-shop.ru/catalog/17a89fea16404e77/sotovye-telefony/?p={page}&order=4&groupBy=none&stock=2'
#     url = f'https://www.dns-shop.ru/catalog/17a8a01d16404e77/smartfony//?p={page}&order=4&groupBy=none&stock=2'

    req = requests.get(url)
    parser = bs4.BeautifulSoup(req.text)
    items = parser.findAll('a', attrs={'data-role': 'product-card-link', 'class': 'show-popover'})
    for item in items:
        links.append(item['href'])

print(links)

['/product/50741fd75a813361/sotovyj-telefon-lg-g360-serebristyj/', '/product/54251da8decc3330/sotovyj-telefon-nokia-230-ds-belyj/', '/product/54251da7decc3330/sotovyj-telefon-nokia-230-ds-cernyj/', '/product/6906ad5b460b3361/sotovyj-telefon-texet-tm-513r-cernyj/', '/product/6fa720e309f33330/sotovyj-telefon-aceline-fl1-belyj/', '/product/408dc2c109f33330/sotovyj-telefon-aceline-fl1-cernyj/', '/product/5c03252d5fdb3330/sotovyj-telefon-dexp-larus-m9-cernyj/', '/product/caa040cebd623330/sotovyj-telefon-dexp-c241-cernyj/', '/product/cacb1c5aaf283330/sotovyj-telefon-dexp-hx10-osmium-cernyj/', '/product/e3307431cb523330/sotovyj-telefon-dexp-fp20-cernyj/', '/product/8944219686213330/sotovyj-telefon-nokia-130-ds-ta-1017-new-krasnyj/', '/product/8944219586213330/sotovyj-telefon-nokia-130-ds-ta-1017-new-seryj/', '/product/8944219486213330/sotovyj-telefon-nokia-130-ds-ta-1017-new-cernyj/', '/product/486d76b42e6e3332/sotovyj-telefon-philips-e580-seryj/', '/product/6c815b51a6ee3330/sotovyj-telefon-d

In [43]:
reviews_page = []
for link in links:
    page_link = f'https://www.dns-shop.ru{link}opinion/'
    req = requests.get(page_link)
    parser = bs4.BeautifulSoup(req.text)
    last_page = parser.select_one("li.last > a")
    if last_page:

        last = int(last_page['data-page'])+1
        reviews_page += [page_link+str(p) for p in range(1, last+1)]

    else:
        reviews_page += [page_link]    


In [44]:
len(reviews_page)

368

In [45]:
# all_reviews = []
# labels_train = []
for page in reviews_page:
    req = requests.get(page)
    parser = bs4.BeautifulSoup(req.text)
    reviews = parser.select("div.opinion-item")
    for review in reviews:
        rating = int(review['data-grade'])
        text_review = review.select('div > div.descriptions > ul > li')
        text_review = ' '.join([element.select_one('span.description-text').text for element in text_review if element.select_one('span.description-text') is not None])
        all_reviews.append(text_review)
        labels_train.append(1 if rating >= 4 else 0)

In [64]:
len(all_reviews)

1643

In [47]:
# ps = nltk.stem.PorterStemmer()
# lemmatizer = nltk.stem.WordNetLemmatizer() 

In [65]:
vectorizer = TfidfVectorizer(ngram_range=(1,3), stop_words=['-', '1.', '2.', '3.', '4.'])
# X = vectorizer.fit_transform([' '.join([ps.stem(word) for word in text.split()]) for text in all_reviews])
X = vectorizer.fit_transform(all_reviews)
clf = LinearSVC()
clf.fit(X, labels_train)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [66]:
with open('test.csv', "r", encoding='utf-8') as input_file:
    data = input_file.read()

test_texts = []
lines = data.split(sep='\n\n')
for line in lines:
    new_line = line.strip().replace('<review>', '').replace('</review>', '').replace('\n', ' ')
    if new_line != '':
        test_texts.append(new_line)
test_texts = np.array(test_texts)

In [67]:
# X_test = vectorizer.transform([' '.join([ps.stem(word) for word in text.split()]) for text in test_texts])
X_test = vectorizer.transform(test_texts)

In [68]:
predicts = clf.predict(X_test)

In [69]:
predicts

array([0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0])

In [71]:
predicts = ['pos' if predict == 1 else 'neg' for predict in predicts]
pd.DataFrame({'reviews': test_texts, 'class': predicts})

,reviews,class
0,"Ужасно слабый аккумулятор, это основной минус ...",neg
1,ценанадежность-неубиваемостьдолго держит батар...,pos
2,"подробнее в комментариях К сожалению, факт пол...",neg
3,я любительница громкой музыки. Тише телефона у...,neg
4,"Дата выпуска - 2011 г, емкость - 1430 mAh, тех...",pos
5,- Удобная Клавиатура и русская раскладка - 2 с...,pos
6,Супер телефон! 1.QWERTY!!! Это самый лучший ег...,pos
7,- толщина (помещается даже в брюки) - аккумуля...,neg
8,Аккумулятор ужасен! Хватает буквально на неско...,neg
9,1 удобный.клавеатура просто класс быстро пишеш...,pos
